In [18]:
from px4_offboard_py.PPC_controller import PPCController
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.spatial.transform import Rotation as R


ppc = PPCController()
state = {
    'pos': np.array([0.0, 0.0, 0.0]),
    'vel': np.array([0.0, 0.0, 0.0]),
    'euler': np.array([0.0, 0.0, 0.0]),
    'omega': np.array([0.0, 0.0, 0.0]),
    'yaw': 0.0
}

t = 0
x = np.cos(t)/(1 + np.cos(t)**2)
y = np.sin(t) * x
z = - t/5 - 1

target_pos = [x, y, z]
print("target_pos =", target_pos)
target_yaw = 0.0
current_pos = np.array(state['pos'])
current_vel = np.array(state['vel'])
current_euler = np.array(state['euler'])
current_omega = np.array(state['omega'])
current_yaw = state['yaw']

e_pos =  target_pos - current_pos
#print ("e_pos =", e_pos )
rho_pos = np.array ([
            ppc.rho_calculation('position', axis, t) for axis in ['px', 'py', 'pz']
             ])
rho_pos_diag = np.diag(rho_pos)
#print ("rho_pos =", rho_pos)
xi_pos =  np.linalg.inv(rho_pos_diag) @ e_pos
eps_pos = ppc.transformation(e_pos, rho_pos)

print("xi =", e_pos / rho_pos)
print("epsilon_pos =", eps_pos)

r_pos = ppc.transformation_derivative(xi_pos)
#print("r_pos =", r_pos)

kp = np.diag([1.25, 1.25, 12.5])         # shape: (3,3)
rho_inv_pos = np.linalg.inv(rho_pos)          # shape: (3,3)

v_ref = -kp @ rho_inv_pos @ r_pos @ eps_pos  # shape: (3,)

print("v_ref =", v_ref)

#velocity layer
e_vel =  v_ref - current_vel 
rho_vel = np.array ([
            ppc.rho_calculation('velocity', axis, t) for axis in ['vx', 'vy', 'vz']
             ])
#print ("rho_vel =", rho_vel)
xi_vel = e_vel / rho_vel
eps_vel = ppc.transformation(e_vel, rho_vel)
r_vel = ppc.transformation_derivative(xi_vel)
#print ("rho_vel =", rho_vel, "xi_vel", xi_vel,"eps_vel",eps_vel, "r_vel", r_vel)

rho_vz_inv = 1.0 / rho_vel[2]
r_vz = r_vel[2, 2]  
eps_vz = eps_vel[2]
# print("rho_vz_inv =", rho_vz_inv)
# print("r_vz =", r_vz)
# print("eps_vz =", eps_vz)

kvz = 10.0
Fz_ref = -kvz * rho_vz_inv * r_vz * eps_vz 
print("Fz_ref =", Fz_ref)

rho_vel_xy_inv = np.diag(1 / rho_vel[0:2])
epsilon_vxy = eps_vel[0:2]
r_vxy = r_vel[:2, :2]   
kvxy = np.diag([1.0, 2.0])

kphitheta = np.diag([3.0, 1.5])
kpsi = 1.0
komega = np.eye(3) * 10.0

phi, theta, psi = current_euler
Cphi, Sphi = np.cos(phi), np.sin(phi) 
Ctheta, Stheta = np.cos(theta), np.sin(theta)
Cpsi, Spsi = np.cos(psi), np.sin(psi)
Ttheta = Stheta / (Ctheta + 1e-6)

R_psi = np.array([
            [Cpsi, -Spsi],
            [Spsi, Cpsi]
            ]) 
T_phitheta_r = -kvxy @ R_psi.T @ rho_vel_xy_inv @ r_vxy @ epsilon_vxy #/ Fz_ref
print("T_phitheta_r =", T_phitheta_r)

current_T_phitheta = np.array([Stheta*Cphi, -Sphi])
e_T_phitheta = T_phitheta_r - current_T_phitheta 

e_yaw =  target_yaw -psi 
e_angular = np.concatenate((e_T_phitheta, [e_yaw]))
rho_angular = np.array ([
    ppc.rho_calculation('angular', axis, t) for axis in ['phitheta1', 'phitheta2', 'psi']
     ])

xi_angular = e_angular / rho_angular
r_angular = ppc.transformation_derivative(xi_angular)
epsilon_angular = ppc.transformation(e_angular, rho_angular)

R_phitheta = np.array([[Cpsi/Ctheta, Spsi/Ctheta],
                           [-Spsi, Cpsi]
                           ])
J_phitheta = np.array([[-Stheta*Sphi, Ctheta*Cphi],
                            [-Cphi, 0]
                            ])
rho_inv_phitheta = np.diag(1.0 / rho_angular[0:2])

r_phitheta = r_angular[:2,:2]
print("AAAAAAAAAAAA",r_angular,r_phitheta )
eps_phitheta = epsilon_angular[0:2]
A = kphitheta @ np.linalg.inv(R_phitheta) @ np.linalg.inv(J_phitheta) @ rho_inv_phitheta @ r_phitheta @ eps_phitheta
B =  kpsi * (1 / rho_angular[2]) * r_angular[2, 2]  * epsilon_angular[2] + current_omega[0] * Cpsi * Ttheta + current_omega[1]* Spsi * Ttheta
omega_ref = -np.concatenate((A,[B]))
print("omega_ref =",omega_ref)

e_omega = omega_ref - current_omega 

rho_omega = np.array ([
            ppc.rho_calculation('omega', axis, t) for axis in ['omega_phi', 'omega_theta', 'omega_psi']
             ])

xi_omega = e_omega / rho_omega
r_omega = ppc.transformation_derivative(xi_omega)
epsilon_omega = ppc.transformation(e_omega, rho_omega)

rho_omega_inv = np.diag(1.0 / rho_omega)          # shape: (3,3)
tau = - komega @ rho_omega_inv @ r_omega @ epsilon_omega

print("tau = ", tau)



target_pos = [0.5, 0.0, -1.0]
xi = [ 0.04166667  0.         -0.08333333]
epsilon_pos = [ 0.0416908   0.         -0.08352704]


LinAlgError: 1-dimensional array given. Array must be at least two-dimensional